# Define hyper-params

In [ ]:
# model param
model_name = "model01"

# train param
epochs = 100
batch_size = 1
optimizer_name = ["Adam", "SGD"][0]
lr = 1e-3
weight_decay = 1e-10

# other settings
import torch
cuda = True and torch.cuda.is_available()
gpu_ids = (0)


print("[Model param]")
print("model_name:", model_name)
print()
print("[Train param]")
print("epochs:", epochs)
print("batch_size:", batch_size)
print("optimizer_name:", optimizer_name)
print("lr:", lr)
print("weight_decay:", weight_decay)
print()
print("[Other setting]")
print("cuda:", cuda)
print("gpu_ids:", gpu_ids)

# Prepare Data

In [ ]:
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from dataloader.dataset import Dataset

%matplotlib inline

## Make datasets

In [ ]:
trainset = Dataset("train")
valset = Dataset("val")
testset = Dataset("test")
num_class = trainset.NUM_CLASSES

### Check datasets

In [ ]:
for i in range(4):
    plt.subplot(1,4,i+1)
    im = trainset[i]["input"].numpy()
    im = im.transpose(1,2,0)
    plt.imshow(im)

In [ ]:
loop = {"train": trainset, "val": valset, "test": testset}
i=0
for split, sets in loop.items():
    i += 1
    print(split,":")
    labels = {}
    try:
        for data in sets:
            label = data["label"].item()
            try:
                labels[label] += 1
            except:
                labels[label] = 1
    except:
        print("None")
        continue
                
    for key in labels.keys():
        print("label [{}] is {}.".format(key, labels[key]))
    plt.subplot(1,3,i)
    plt.subplots_adjust(wspace=0.4)
    plt.bar(range(len(labels)), labels.values())
    plt.title(split)
    plt.xticks(range(len(labels)), labels.keys())
    print()

## Make DataLoaders

In [ ]:
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)

# Define Saver

In [ ]:
from utils.saver import Saver

In [ ]:
saver = Saver(model_name, lr, epochs)

In [ ]:
saver.save_experiment_config()

# Define Tensorboard Summary

In [ ]:
from utils.summaries import TensorboardSummary

In [ ]:
summary = TensorboardSummary(saver.experiment_dir)
writer = summary.create_summary()

# Define Model

In [ ]:
import gensim
import torch
import torch.nn as nn

from modeling.modeling import Modeling
from config import Config
conf = Config()

cuda = True

In [ ]:
model = Modeling(embedding_dim=conf.embedding_dim,
                 c_out=conf.num_class,
                 c_hidden=conf.hidden_channel,
                 cuda=cuda,
                 hidden_layer=conf.hidden_layer)
word_vector = gensim.models.KeyedVectors.load_word2vec_format(conf.word_vector_dir+'model.vec', binary=False)
model.word_embeddings.weight = nn.Parameter(torch.from_numpy(word_vector.vectors))
for param in model.parameters():
    param.requires_grad = True
model.word_embeddings.weight.requires_grad=False
if cuda:
    model = torch.nn.DataParallel(model, device_ids=(0,))
    model = model.cuda()

In [ ]:
PATH = "./run/model01/experiment_06/checkpoint.pth.tar"
model_state = torch.load(PATH)
state_dict = model_state["state_dict"]
model.load_state_dict(state_dict)

# Define Evaluator

In [ ]:
from utils.metrics import Evaluator

In [ ]:
evaluator = Evaluator(num_class)

# Define Optimizer

In [ ]:
import torch

In [ ]:
if optimizer_name=="Adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
elif optimizer_name=="SGD":
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)

# Define Criterion

In [ ]:
import torch.nn as nn

In [ ]:
criterion = nn.CrossEntropyLoss(reduction="none")

# training

## run epoch

In [ ]:
from tqdm import tqdm_notebook as tqdm

from config import pycolor

In [ ]:
def run_epoch(epoch, best_pred, mode="train"):
    # ------------------------- #
    # Initializing
    epoch_loss = 0.0
    ## Set model mode & tqdm (progress bar; it wrap dataloader)
    assert mode=="train" or mode=="val", "argument 'mode' can be 'train' or 'val.' Not {}.".format(mode)
    if mode=="train":
        print(pycolor.GREEN + "[Epoch: {}]".format(epoch) + pycolor.END)
        print(pycolor.YELLOW+"Training:"+pycolor.END)
        model.train()
        tbar = tqdm(train_loader, leave=False)
        num_dataset = len(train_loader)
    elif mode=="val":
        print(pycolor.YELLOW+"Validation:"+pycolor.END)
        model.eval()
        tbar = tqdm(val_loader, leave=False)
        num_dataset = len(val_loader)
    ## Reset confusion matrix of evaluator
    evaluator.reset()

    # ------------------------- #
    # Run 1 epoch
    for i, sample in enumerate(tbar):
        inputs, target = sample["input"], sample["label"]
        if cuda:
            inputs, target = inputs.cuda(), target.cuda()
        if mode=="train":
            optimizer.zero_grad()
            output = model(inputs)
        elif mode=="val":
            with torch.no_grad():
                output = model(inputs)
        loss = criterion(output, target).sum()
        if mode=="train":
            loss.backward()
            optimizer.step()
        epoch_loss += loss.item()
        tbar.set_description('{} loss: {:.3f}'.format(mode, (epoch_loss / ((i + 1)*batch_size))))
        # Compute Metrics
        pred = output.data.cpu().numpy()
        pred = np.argmax(pred, axis=1)
        target = target.cpu().numpy()
        ## Add batch into evaluator
        evaluator.add_batch(target, pred)

    # ------------------------- #
    # Save Log
    ## **********Evaluate**********
    Acc = evaluator.Accuracy()
    F_score_Average = evaluator.F_score_Average()

    ## Save results
    writer.add_scalar('{}/loss_epoch'.format(mode), epoch_loss / num_dataset, epoch)
    writer.add_scalar('{}/Acc'.format(mode), Acc, epoch)
    writer.add_scalar('{}/F_score'.format(mode), F_score_Average, epoch)
    print('Total {} loss: {:.3f}'.format(mode, epoch_loss / num_dataset))
    print("Acc:{}, F_score:{}".format(Acc, F_score_Average))

    ## Save model
    if mode=="val":
        new_pred = F_score_Average
        print("---------------------")
        if new_pred > best_pred:
            is_best = True
            print("model improve best score from {:.4f} to {:.4f}.".format(best_pred, new_pred))
            best_pred = new_pred
            saver.save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'best_pred': best_pred,
            }, is_best)
    return best_pred

In [ ]:
def run_all():
    best_pred = 0
    for epoch in tqdm(range(epochs), desc="Epochs:"):
        ## ***Train***
        run_epoch(epoch, best_pred, mode="train")
        ## ***Validation***
        best_pred = run_epoch(epoch, best_pred, mode="val")
    writer.close()

In [ ]:
import numpy as np

run_all()